### Main steps
For several song samples:

- load a song stft from data/stft/fft-window-size_fft-hop-length/Album-name/song-name-dir/startoffset-endoffset.stft
- Use the method in [Noll's paper](https://drive.google.com/file/d/104KZa5Zdww6NLlG7VzhKRKxS-sbp0Ej8/view?usp=sharing) to detect the one, two, or three pitches (fundamental frequencies) of the voices in the audio
    - Equation 14 in Noll gives the Cepstrum as an integral. Translated into Librosa functions this is
    - stft(log|stft(signal)|)
    
The idea is that if you take the Fourier transform of a voice you get peaks in the fundamental and harmonics. This looks like a wave with period = fundamental frequency, so you can take the Fourier transform of *it* to get the period. 

    
    

It's not quite that simple. If you take the Fourier transform of the Fourier transform you get the original signal back. 

The vocal cords buzz at a certain frequency. The shape of the space in the throat, mouth, and nose makes it resonate to this frequency and harmonics. Depending on the shape it resonates more or less at different freqencies. Fig. 4 of [Noll's paper](https://drive.google.com/file/d/104KZa5Zdww6NLlG7VzhKRKxS-sbp0Ej8/view) shows the sound of the vowel "A": it has a peak at about 500Hz and another peak at about 1800Hz. The small bumps in the spectrum show the pitch. There are 8-9 bumps per 1000Hz, so the pitch is about 120Hz.

To get Fig. 4 you need to take the Fourier transform of the signal, then take the log of the absolute value of that. In the Fourier transform the contributions of the vowel and the pitch are multiplied together. After taking the log they are added together, which lets us see the pitch and vowel separately in the graph. 

When we transform this new "signal" again using the Fourier transform it picks up the frequency of the bumps. That's the fundamental frequency.

In [ ]:
import librosa
import os
from scipy import signal
from scipy.fftpack import fft, fftshift, ifft
from scipy.optimize import brent, fminbound
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import numpy as np
import math

data_dir = ""
pitch_dir = ""
crepe_dir = ""
correction_dir = ""
maximum_pitch = 2000
minimum_pitch = 60
crepe_interval_estimate = 0.2

def load_file(name):
    y, sr = librosa.load(data_dir + name, sr=None)
    return (y, sr)

def load_crepe(name):
    name = name[:-3] + 'txt'
    t, freq = np.loadtxt(crepe_dir+name)[:,0], np.loadtxt(crepe_dir+name)[:,1]
    return (t, freq)

def load_corrections(name):
    name = name[:-3] + "txt"
    corrections = [] # (t1, p1, t2, p2)
    try: correction_file = open(f'{correction_dir}{name}')
    except: 
        print('no correction file for', name)
        return []
    for line in correction_file:
        current_corrections = list(map(int, line.split(' ')))
        corrections.append(current_corrections)
    return corrections

In [ ]:
def find_correction_box (time, corrections):
    for correction in corrections:
        if correction[0] <= time <= correction[2]:
            return (correction[1], correction[3])

In [ ]:
def local_maxes (a):
    return a[np.r_[False, (a[1:-1] > a[:-2]) & (a[1:-1] >= a[2:]), False]]

tck = None

def cubic_interpolation_init (signal):
    global tck
    x_points = np.arange(0, signal.size)
    tck = interp1d(x_points, -signal, kind="cubic")
    
def get_cubic_interpolation (x):
    global tck
    try:
        return tck(x)
    except:
        return 0
    
def find_max_cubic_interpolation (l, r):
    global tck
    output = fminbound(get_cubic_interpolation, l, r, full_output=True)
    return [output[0], -output[1]]

def find_fundamental_frequency (y, sr, start_time, crepe_estimate, correction_box, graph=False):
    #generate cepstral
    window_size = int(np.ceil(0.030 * sr))
    segment = y[np.int(start_time * sr) : np.int(start_time * sr + window_size)]
    window = signal.hann(window_size)[:segment.size]
    prod = np.multiply(segment, window)
    A = fft(prod)
    response = np.log(np.abs(fftshift(A)))
    final = np.real(ifft(np.multiply(response, window)))**2
    final = final[:final.size//2]
    
    # minimize values near zero
    final[0:np.int(0.0005 * sr)] = 0
    
    # find range
    mx, my = (0, 0)
    analysis_range = final[np.int(0.0005 * sr):]
    maxes = np.flip(np.sort(local_maxes(analysis_range)))
    
    # initialize cubic interpolation
    cubic_interpolation_init(analysis_range)
    
    for i, j in enumerate(analysis_range):
        current_estimate = float(sr)/(i+np.int(0.0005*sr))
        current_bound = crepe_interval_estimate*crepe_estimate
        within_bound = np.abs(current_estimate-crepe_estimate) <= current_bound
        within_bound = within_bound or crepe_estimate <= 0 
        
        if not correction_box is None: # if there is a correction box, override crepe estimate
            within_bound = correction_box[0] <= current_estimate <= correction_box[1]
        
        if j > my and within_bound:
            mx = i
            my = j
            
    nx, ny = find_max_cubic_interpolation(mx-1, mx+1)
    
    if graph:
        # the range between the purple lines is what is considered for frequency
        plt.figure(figsize=(15, 3))
        plt.xlabel("Quefrency (ms)")
        plt.ylabel("Amplitude")
        plt.plot(final[:1000])   

        # the max peak is found inside the range set by these lines
        plt.axvline(x=0.0005 * sr, color="purple")
        plt.axvline(x=0.015 * sr, color="purple")
        plt.axvline(x=(mx + np.int(0.0005 * sr)), color='green')
        plt.axvline(x=crepe_estimate, color='red')
    
    frequency = float(sr)/(nx + np.int(0.0005 * sr))
    return frequency

In [ ]:
def create_and_store (names):
    done = 1
    for name in names:
        y, sr = load_file(name)
        
        song_dir = pitch_dir + name
        song_dir = song_dir[:-4] + ".txt"
        
#         if os.path.isfile(song_dir):
#             print(name, 'already exists. Skipped')
#             continue
        
        try: ct, cfreq = load_crepe(name)
        except: continue
        corrections = load_corrections(name)
        
        time_listing = np.arange(0, y.size/sr, 0.01)[:-1]
        f = np.empty(time_listing.size)
        for i, t in enumerate(time_listing):
            current_box = find_correction_box(t, corrections)
            if i >= len(cfreq): 
                f[i] = find_fundamental_frequency(y, sr, t, 0, current_box, graph=False)
            else: 
                f[i] = find_fundamental_frequency(y, sr, t, cfreq[i], current_box, graph=False)
            
        threshold = (f > maximum_pitch) ^ (f < minimum_pitch)
        f[threshold] = 0
        
        f = f.tolist()
        for i in range(len(f)): f[i] = str(time_listing[i]) + ' ' + str(f[i])
                    
        song_dir = pitch_dir + name
        song_dir = song_dir[:-4] + ".txt"
        
        fout = open(song_dir, "w+")
        fout.write("\n".join(f))
        fout.close()
        
        print(f"{done} done")
        done += 1

In [ ]:
# Scherbaum Mshavandaze
parent_data_dir = '/Akamai/voice/data/Scherbaum Mshavanadze/'
parent_pitch_dir = '/Akamai/voice/data/pitches-raw-crepe-assisted/noll/Scherbaum Mshavanadze/'
parent_crepe_dir = '/Akamai/voice/data/pitches-vuv-new/crepe/Scherbaum Mshavanadze/'
parent_correction_dir = '/Akamai/voice/data/pitch-corrections/crepe/Scherbaum Mshavanadze/'

for collection in os.listdir(parent_data_dir):
    if os.path.isdir(f"{parent_data_dir}{collection}") and '19' in collection:
        parts = []
        for part in os.listdir(f"{parent_data_dir}{collection}"):
            if part[-3:] == 'wav':
                parts.append(part)

        data_dir = parent_data_dir + collection + '/'
        pitch_dir = parent_pitch_dir + collection + '/'
        crepe_dir = parent_crepe_dir + collection + '/'
        correction_dir = parent_correction_dir + collection + '/'
        create_and_store(parts)
        break
        
# Teach Yourself Megrelian Songs
# parent_data_dir = '/Akamai/voice/data/Teach Yourself Megrelian Songs/'
# parent_pitch_dir = '/Akamai/voice/data/pitches-raw-crepe-assisted/noll/Teach Yourself Megrelian Songs/'
# parent_crepe_dir = '/Akamai/voice/data/pitches-vuv-new/crepe/Teach Yourself Megrelian Songs/'
# parent_correction_dir = '/Akamai/voice/data/pitch-corrections/crepe/Teach Yourself Megrelian Songs/'

# for collection in os.listdir(parent_data_dir):
#     if os.path.isdir(f"{parent_data_dir}{collection}"):
#         if collection != 'mp3':
#             parts = []
#             for part in os.listdir(f"{parent_data_dir}{collection}"):
#                 if part[-3:] == 'wav':
#                     parts.append(part)

#             data_dir = parent_data_dir + collection + '/'
#             pitch_dir = parent_pitch_dir + collection + '/'
#             crepe_dir = parent_crepe_dir + collection + '/'
#             correction_dir = parent_correction_dir + collection + '/'

#             create_and_store(parts)
            
# Teach Yourself Gurian Songs
# parent_data_dir = '/Akamai/voice/data/Teach Yourself Gurian Songs/'
# parent_pitch_dir = '/Akamai/voice/data/pitches-raw-crepe-assisted/noll/Teach Yourself Gurian Songs/'
# parent_crepe_dir = '/Akamai/voice/data/pitches-vuv-new/crepe/Teach Yourself Gurian Songs/'
# parent_correction_dir = '/Akamai/voice/data/pitch-corrections/crepe/Teach Yourself Gurian Songs/'


# for collection in os.listdir(parent_data_dir):
#     if os.path.isdir(f"{parent_data_dir}{collection}"):
#         if collection != 'mp3':
#             parts = []
#             for part in os.listdir(f"{parent_data_dir}{collection}"):
#                 if part[-3:] == 'wav':
#                     parts.append(part)

#             data_dir = parent_data_dir + collection + '/'
#             pitch_dir = parent_pitch_dir + collection + '/'
#             crepe_dir = parent_crepe_dir + collection + '/'
#             correction_dir = parent_correction_dir + collection + '/'
#             create_and_store(parts)